In [11]:
# Para llevar un registro de los cambios
%pip install -q comet_ml
import comet_ml
comet_ml.init(project_name='Tarea 03')

In [12]:
# Librerías
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [13]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)
experiment.set_name("Sobre_ajuste_1")

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/a1e87c4f8a024e49a3f223a5c6111468
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [3]                  : (0.902816653251648, 0.9641000032424927)
COMET INFO:     batch_accuracy [3702]         : (0.12727272510528564, 1.0)
COMET INFO:     batch_loss [3702]             : (0.048914890736341476, 2.5921216011047363)
COMET INFO:     epoch_duration [3]            : (117.92520224799995, 126.33244855100003)
COMET INFO:     loss [3]                      : (0.11817961186170578, 0.31512555480003357)
COMET INFO:     val_accuracy [3]              : (0.9581999778747559, 0.9729999899864197)
COMET INFO:    

In [14]:
# Carga el dataset de MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [15]:
# Convierte y normaliza los datos de entrada
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [16]:
# to_categorical convierte las etiquetas (train, test) en matrices one-hot
num_classes = 10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [17]:
  # Diccionario con los parámetros que se registrarán en comet
  parameters = {
      'epochs' : 20,
      'batch_size' : 10,
      'learning_rate' : 0.010,
      'loss' : 'categorical_crossentropy',
      'optimizer' : 'adam'
  }
  experiment.log_parameters(parameters)

In [18]:
# Creación del modelo
model = Sequential()
# Es como una "primera capa" de tres capas
model.add(Input(shape = (28,28))) # Matrices de entrada de 28x28 (imáegenes)
model.add(Flatten()) # Convierte las matrices de 28x28 en matrices de 784x1
model.add(Dense(784, activation = 'sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(30, activation = 'relu')) # Segunda capa
model.add(Dense(10, activation = 'softmax')) # Tercera capa

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 784)               615440    
                                                                 
 dropout_1 (Dropout)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 30)                23550     
                                                                 
 dense_5 (Dense)             (None, 10)                310       
                                                                 
Total params: 639300 (2.44 MB)
Trainable params: 639300 (2.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Checkpoint que guarda el mejor modelo (cuando la función de costo alcanza un mínimo)
checkpoint = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'min')

In [20]:
# optimizer = RMSprop(learning_rate = 0.0005)
optimizer = 'adam'
model.compile(loss = parameters['loss'], optimizer = optimizer, metrics = ['accuracy'])
model.fit(x_train, y_trainc, batch_size = parameters['batch_size'], epochs = parameters['epochs'], verbose = 1, validation_data = (x_test, y_testc), callbacks = [checkpoint])

Epoch 1/20
6000/6000 [==============================] - ETA: 0s - loss: 0.3388 - accuracy: 0.8963
Epoch 1: val_loss improved from inf to 0.14820, saving model to best_model.hdf5
6000/6000 [==============================] - 72s 12ms/step - loss: 0.3388 - accuracy: 0.8963 - val_loss: 0.1482 - val_accuracy: 0.9552
Epoch 2/20
5999/6000 [============================>.] - ETA: 0s - loss: 0.1587 - accuracy: 0.9527
Epoch 2: val_loss improved from 0.14820 to 0.09813, saving model to best_model.hdf5
6000/6000 [==============================] - 65s 11ms/step - loss: 0.1587 - accuracy: 0.9527 - val_loss: 0.0981 - val_accuracy: 0.9704
Epoch 3/20
5995/6000 [============================>.] - ETA: 0s - loss: 0.1172 - accuracy: 0.9652
Epoch 3: val_loss improved from 0.09813 to 0.08460, saving model to best_model.hdf5
6000/6000 [==============================] - 63s 10ms/step - loss: 0.1172 - accuracy: 0.9653 - val_loss: 0.0846 - val_accuracy: 0.9753
Epoch 4/20
5996/6000 [============================>.]

In [21]:
experiment.log_model('MNIST1', 'best_model.hdf5')
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/2dc93145db244648aca7daacd96f56e3
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [20]                  : (0.896316647529602, 0.9906499981880188)
COMET INFO:     batch_accuracy [12000]         : (0.12727272510528564, 1.0)
COMET INFO:     batch_loss [12000]             : (1.1598736818996258e-05, 2.563690662384033)
COMET INFO:     epoch_duration [20]            : (58.972847939000076, 72.03461622099996)
COMET INFO:     loss [20]                      : (0.029687199741601944, 0.338828980922699)
COMET INFO:     val_accuracy [20]              : (0.9552000164985657, 0.9843999743461609)
COMET INFO